In [52]:
import pandas as pd
import sqlite3
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [53]:
game_log = pd.read_csv('game_log.csv')
park_codes = pd.read_csv('park_codes.csv')
person_codes = pd.read_csv('person_codes.csv')
team_codes = pd.read_csv('team_codes.csv')

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
dataframes = {'game_log': game_log, 'park_codes': park_codes, 'person_codes': person_codes, 'team_codes': team_codes}

In [55]:
for key, val in dataframes.items():
    print(val.shape)
    print(val.head(3))

(171907, 161)
       date  number_of_game day_of_week v_name v_league  v_game_number h_name  \
0  18710504               0         Thu    CL1      NaN              1    FW1   
1  18710505               0         Fri    BS1      NaN              1    WS3   
2  18710506               0         Sat    CL1      NaN              2    RC1   

  h_league  h_game_number  v_score  h_score  length_outs day_night completion  \
0      NaN              1        0        2         54.0         D        NaN   
1      NaN              1       20       18         54.0         D        NaN   
2      NaN              1       12        4         54.0         D        NaN   

  forefeit protest park_id  attendance  length_minutes v_line_score  \
0      NaN     NaN   FOR01       200.0           120.0    000000000   
1      NaN     NaN   WAS01      5000.0           145.0    107000435   
2      NaN     NaN   RCK01      1000.0           140.0    610020003   

  h_line_score  v_at_bats  v_hits  v_doubles  v_tri

In [56]:
def run_command(query):
    with sqlite3.connect('ml.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        return conn.execute(query)

In [57]:
def run_query(query):
    with sqlite3.connect('ml.db') as conn:
        return pd.read_sql(query, conn)

In [58]:
with sqlite3.connect('ml.db') as conn:
    for key, val in dataframes.items():
        run_command('DROP TABLE IF EXISTS ' + key)
        val.to_sql(key, conn, index=False)

In [59]:
run_command('''
ALTER TABLE game_log
    ADD game_id INTEGER
''')

In [60]:
run_query('SELECT game_id FROM game_log LIMIT 4')

,game_id
0,None
1,None
2,None
3,None


In [61]:
run_command('''
UPDATE game_log
    SET game_id = (
        SELECT (h_name || date || number_of_game)
        FROM game_log
    )
''')

In [62]:
run_query('SELECT game_id FROM game_log LIMIT 4')

,game_id
0,FW1187105040
1,FW1187105040
2,FW1187105040
3,FW1187105040


# New table schema

![New Schema](https://s3.amazonaws.com/dq-content/193/mlb_schema.svg)

### Create person table

In [63]:
c = '''
CREATE TABLE IF NOT EXISTS person (
    person_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
)
'''

run_command(c)

In [64]:
c = '''
INSERT OR IGNORE INTO person
    SELECT
        id,
        first,
        last
    FROM person_codes
'''
run_command(c)

In [65]:
run_query('SELECT * FROM person LIMIT 4')

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase


### Create park table

In [66]:
c = '''
CREATE TABLE IF NOT EXISTS park (
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
)
'''
run_command(c)

In [67]:
c = '''
INSERT OR IGNORE INTO park
    SELECT
        park_id,
        name,
        aka,
        city,
        state,
        notes
    FROM park_codes
'''
run_command(c)

In [68]:
run_query('SELECT * FROM park LIMIT 4')

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None


### Create league table

In [69]:
c = '''
CREATE TABLE IF NOT EXISTS league (
    league_id TEXT PRIMARY KEY,
    name TEXT
)
'''
run_command(c)

In [70]:
c = '''
INSERT OR IGNORE INTO league
    VALUES 
        ('NL', 'National League'),
        ('AL', 'American League'),
        ('AA', 'American Association'),
        ('FL', 'Federal League'),
        ('PL', 'Players League'),
        ('UA', 'Union Association')
'''
run_command(c)

In [71]:
run_query('SELECT * FROM league')

,league_id,name
0,NL,National League
1,AL,American League
2,AA,American Association
3,FL,Federal League
4,PL,Players League
5,UA,Union Association


### Create appearance_type table

In [72]:
c = '''
CREATE TABLE IF NOT EXISTS appearance_type (
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT
)
'''
run_command(c)

In [73]:
app_types = pd.read_csv('appearance_type.csv')
with sqlite3.connect('ml.db') as conn:
    app_types.to_sql('appearance_type', conn, index=False, if_exists='append')

In [74]:
run_query('SELECT * FROM appearance_type')

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


### Create team table

In [75]:
c = '''
CREATE TABLE IF NOT EXISTS team (
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) REFERENCES league(league_id)
)
'''
run_command(c)

In [76]:
c = '''
INSERT OR IGNORE INTO team
    SELECT
        team_id,
        league,
        city,
        nickname,
        franch_id
    FROM team_codes
'''
run_command(c)

In [77]:
run_query('SELECT * FROM team LIMIT 5')

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


### Create game table

In [84]:
c = '''
CREATE TABLE IF NOT EXISTS game (
    game_id TEXT PRIMARY KEY,
    date TEXT,
    number_of_game INTEGER,
    park_id TEXT,
    length_outs INTEGER,
    day BOOLEAN,
    completion TEXT,
    forefeit TEXT,
    protest TEXT,
    attendance INTEGER,
    legnth_minutes INTEGER,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES park(park_id)
)
'''
run_command(c)

In [85]:
c = '''
INSERT OR IGNORE INTO game
    SELECT
        game_id,
        date,
        number_of_game,
        park_id,
        length_outs,
        CASE
            WHEN day_night = "D" THEN 1
            WHEN day_night = "N" THEN 0
            ELSE NULL
            END
            AS day,
        completion,
        forefeit,
        protest,
        attendance,
        length_minutes,
        additional_info,
        acquisition_info
    FROM game_log
'''
run_command(c)

In [86]:
run_query('SELECT * FROM game LIMIT 5')

,game_id,date,number_of_game,park_id,length_outs,day,completion,forefeit,protest,attendance,legnth_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54,1,None,None,None,200,120,None,Y


In [90]:
c = '''
CREATE TABLE IF NOT EXISTS team_appearance (
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id)
)
'''
run_command(c)

### Create team_appearance table

In [91]:
c = '''
INSERT OR IGNORE INTO team_appearance
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    FROM game_log

UNION

    SELECT    
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    FROM game_log;
'''
run_command(c)  

In [92]:
run_query('SELECT * FROM team_appearance LIMIT 2')

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ALT,FW1187105040,0,UA,1,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ANA,FW1187105040,0,AL,0,000000000,26.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,9.0,0.0,0.0,2.0,0.0,1.0,3.0,5.0,5.0,0.0,0.0,24.0,9.0,1.0,0.0,0.0,0.0


### Create person_appearance table

In [94]:
c = '''
CREATE TABLE person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
)
'''
run_command(c)

In [95]:
c = '''
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL    

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE "1b_umpire_id" IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL;
'''
run_command(c)

In [97]:
t_string = '''
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {ind}_name,
        {ind}_player_{i}_id,
        "O{i}"
    FROM game_log
    WHERE {ind}_player_{i}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {ind}_name,
        {ind}_player_{i}_id,
        "D" || CAST({ind}_player_{i}_def_pos AS INT)
    FROM game_log
    WHERE {ind}_player_{i}_id IS NOT NULL;
'''

for ind in ['h', 'v']:
    for i in range(1,10):
        query_vars = {
            'ind': ind,
            'i': i
        }
        run_command(t_string.format(**query_vars))

In [99]:
run_query('SELECT * FROM person_appearance LIMIT 5')

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,1,abbec101,None,FW1187105040,U1B
1,2,abbof101,None,FW1187105040,UHP
2,3,adamj901,None,FW1187105040,UHP
3,4,addyb101,None,FW1187105040,UHP
4,5,adleu901,None,FW1187105040,UHP


### Drop non-normalized tables

In [100]:
for key, val in dataframes.items():
    run_command('DROP TABLE ' + key)

In [106]:
run_query('SELECT name FROM sqlite_master WHERE type = "table"')

,name
0,person
1,park
2,league
3,appearance_type
4,team
5,game
6,team_appearance
7,person_appearance


### Done!